# Installing Dependencies: Kaolin and PyTorch3D

In [ ]:
!pip install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 --quiet

     |██████████████▌                 | 834.1 MB 1.2 MB/s eta 0:14:05tcmalloc: large alloc 1147494400 bytes == 0x38eda000 @  0x7f71e9b1b615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |██████████████████▍             | 1055.7 MB 1.2 MB/s eta 0:11:06tcmalloc: large alloc 1434370048 bytes == 0x7d530000 @  0x7f71e9b1b615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |███████████████████████▎        | 1336.2 MB 97.8 MB/s eta 0:00:06tcmalloc: large alloc 1792966656 bytes == 0x2362000 @  0x7f71e9b1b615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
INSTALL_PATH = Path("/content/drive/'MyDrive'/CV_DMTet/")
%cd  $INSTALL_PATH

Mounted at /content/drive
/content/drive/MyDrive/CV_DMTet


In [ ]:
# reinstall cython, install usd-core (for 3D rendering), and clone into kaolin repo
!pip uninstall Cython --yes
!pip install Cython==0.29.20  --quiet
!pip install usd-core --quiet

Found existing installation: Cython 0.29.32
Uninstalling Cython-0.29.32:
  Successfully uninstalled Cython-0.29.32
     |████████████████████████████████| 1.9 MB 4.3 MB/s 
     |████████████████████████████████| 24.7 MB 3.9 MB/s 


In [ ]:
# installing kaolin and check version
%env IGNORE_TORCH_VER=1
%env KAOLIN_INSTALL_EXPERIMENTAL=1
KAOLIN_PATH = INSTALL_PATH / "kaolin"
%cd $INSTALL_PATH
!if [ ! -d $KAOLIN_PATH ]; then git clone --recursive https://github.com/NVIDIAGameWorks/kaolin; fi;
%cd $KAOLIN_PATH
SETUP_CHECK = KAOLIN_PATH / "kaolin" / "version.py"
# !echo Checking if $SETUP_CHECK exists
# !if [ ! -f $SETUP_CHECK ]; then python setup.py develop; fi;
!python setup.py develop
!python -c "import kaolin; print(kaolin.__version__)"

# Import packages

In [ ]:
import numpy as np
import torch
import kaolin
import sys
import os

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

from kaolin.ops.conversions import (
    trianglemeshes_to_voxelgrids,
    marching_tetrahedra,
    voxelgrids_to_cubic_meshes,
    voxelgrids_to_trianglemeshes,
)

from kaolin.ops.mesh import (
    index_vertices_by_faces
)

from kaolin.io.shapenet import (
    ShapeNetV2
)

from kaolin.metrics.trianglemesh import (
    point_to_mesh_distance,

)

from torch.utils.data import DataLoader

# add path for demo utils functions 
sys.path.append(os.path.abspath(''))
sys.path.append('/content/drive/MyDrive/CV_DMTet/')
sys.path.append('/content/drive/MyDrive/CV_DMTet/pvcnn')

torch.manual_seed(3407)
torch.cuda.manual_seed(3407)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.7 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221122-py3-none-any.whl size=61484 sha256=8678fd5f70d0233ca0fd5dc86b875d94dd83be81d87c6d7375d0650242e273a5
  Stored in directory: /root/.cache/pip/wheels/15/6e/e3/602889ca9c5c55020f8d205066445ac5b1b96df59f75170ca0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=cf8b6d9d38529d9463cfcc770559db88b91006562906a9da70f49588502519e1
  Stored in directory: /root/.cache/pip/wheels/89/3e/24/0f349c0b2eeb6965903035f3b00dbb5c9bea437b4a2f18d82c
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py38_cu113_pyt1121/download.html
     |████████████████████████████████| 47.2 MB 98.6 MB/s 


# Import Dataset: Subset of ShapeNetV2

In [ ]:
import pytorch3d

In [ ]:
state_dict = torch.load('/content/drive/MyDrive/CV_DMTet/shapenet.pvcnn.c1.pth.tar')
state_dict.keys()

dict_keys(['model'])

In [ ]:
sys.path.append('/content/drive/MyDrive/kaolin/examples/')
sys.path.append('/content/drive/MyDrive/kaolin/examples/tutorial')

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
device

device(type='cuda', index=0)

In [ ]:
# path to the point cloud to be reconstructed
pcd_path = "/content/drive/MyDrive/kaolin/examples/samples/bear_pointcloud.usd"
# path to the output logs (readable with the training visualizer in the omniverse app)
logs_path = '/content/drive/MyDrive/CV_DMTet/Logs'

# We initialize the timelapse that will store USD for the visualization apps
timelapse = kaolin.visualize.Timelapse(logs_path)

#Load Tetrahedral grid

DMTet starts from a uniform tetrahedral grid of predefined resolution, and uses a network to predict the SDF value as well as deviation vector at each grid vertex.

Here we load the pre-generated tetrahedral grid using Quartet at resolution 128, which has roughly the same number of vertices as a voxel grid of resolution 65. We use a simple MLP + positional encoding to predict the SDF and deviation vectors in DMTet, and initialize the encoded SDF to represent a sphere.

In [ ]:
# Uniform Tetrahedral Grid
tets_verts = torch.tensor(np.load('/content/drive/MyDrive/kaolin/examples/samples/128_verts.npz')['data'], dtype=torch.float, device=device)
tets = torch.tensor(([np.load('/content/drive/MyDrive/kaolin/examples/samples/128_tets_{}.npz'.format(i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)
print(tets_verts, tets)

tensor([[ 0.5000,  0.5000,  0.4844],
        [ 0.4844,  0.5000,  0.4922],
        [ 0.4922,  0.4844,  0.4844],
        ...,
        [-0.1719, -0.5000,  0.4766],
        [-0.1562, -0.5000,  0.4688],
        [-0.1562, -0.4922,  0.4688]], device='cuda:0') tensor([[     0,      1,      2,      3],
        [     2,      3,      1,      4],
        [     5,      3,      0,      2],
        ...,
        [277409, 272920, 272914, 272919],
        [272919, 277409, 272920, 274866],
        [277409, 277400, 272920, 274866]], device='cuda:0')


<ipython-input-11-6909fe0ff698>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tets = torch.tensor(([np.load('/content/drive/MyDrive/kaolin/examples/samples/128_tets_{}.npz'.format(i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)


# Loading from ShapeNet

In [ ]:
SHAPENET_PATH = "/content/drive/MyDrive/CV_DMTet/Core"
#SHAPENET_PATH = "/content/drive/MyDrive/FALL 2022/Computer Vision/Project/Core"
SYNSETS_IDS = ['02747177', '02773838', '02801938', '02808440', '02818832', '02828884', '02843684', '02871439', '02876657', '02880940', '02924116', '02933112']
shapenet_train = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True)
shapenet_test = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True, train=False)

In [ ]:
sample_model = shapenet_train[1500] # change the index here for different models
sample_verts = sample_model['mesh'][0]
sample_faces = sample_model['mesh'][1]

timelapse.add_mesh_batch(
    category='gt',
    vertices_list=[sample_verts.cpu()],
    faces_list=[sample_faces.cpu()]
)

# Convert model to watertight meshes

We used a voxelization with resolution of 64 to predict the sdf and extract surface. 

In [ ]:
wt_grid = kaolin.ops.conversions.trianglemeshes_to_voxelgrids(
    vertices=sample_verts.unsqueeze(0).to(device),
    faces=sample_faces.to(device),
    resolution=64
)

In [ ]:
wt_verts, wt_faces = kaolin.ops.conversions.voxelgrids_to_cubic_meshes(wt_grid)
wt_verts, wt_faces = wt_verts[0], wt_faces[0]

In [ ]:
# wt_verts = kaolin.metrics.trianglemesh.uniform_laplacian_smoothing(wt_verts[0].unsqueeze(0), wt_faces[0])
# sample_verts, sample_faces = wt_verts[0], wt_faces[0]

In [ ]:
center = (wt_verts.max(0)[0] + wt_verts.min(0)[0]) / 2
max_l = (wt_verts.max(0)[0] - wt_verts.min(0)[0]).max()
wt_verts = ((wt_verts - center) / max_l)* 0.9
timelapse.add_mesh_batch(
    category='watertight_test',
    vertices_list=[wt_verts.cpu()],
    faces_list=[wt_faces.cpu()]
)

# SDF model

We follow the paper recommandation and use a four-layer
MLPs with hidden dimensions 256, 256, 128 and 64, respectively

In [ ]:
# Since we skip PVCNN, input dimension is just the coordinates of each grid

SDF_MLP_CONFIG = {
    'input_dim' : 3, # Coordinates of the grid's vertices
    'hidden_dims' : [256, 256, 128, 64],
    'output_dim' : 1, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
    'multires': 2
}

lr = 0.01
laplacian_weight = 0.1
iterations = 2000
save_every = 100
multires = 2
grid_res = 128

In [ ]:
sys.path.append('/content/drive/MyDrive/CV_DMTet/pvcnn')  

# from pvcnn.modules.pvconv import PVConv

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary



class MLP(torch.nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()

        self.input_dim = config['input_dim']
        self.hidden_dims  = config['hidden_dims']
        self.output_dim = config['output_dim']
        self.multires = config['multires']

        self.embed_fn = None
        # if self.multires > 0:
        #     embed_fn, input_ch = get_embedder(self.multires)
        #     self.embed_fn = embed_fn
        #     self.input_dim = input_ch

        # Hidden layers
        self.hiddens = nn.ModuleList()
        in_dim = self.input_dim
        for k in range(len(self.hidden_dims)):
            self.hiddens.append(nn.Linear(in_dim, self.hidden_dims[k]))
            in_dim = self.hidden_dims[k]

        # Output layer
        self.output_layer = torch.nn.Linear(self.hidden_dims[-1], self.output_dim)


    def forward(self, x):
        if self.embed_fn is not None:
            x = self.embed_fn(x)
        for hidden in self.hiddens :
            x = F.relu(hidden(x))
        output = self.output_layer(x) # No activation (linear) cuz we do regression

        return output, x # Return sdf predicted + f_v feature vector


# Positional Encoding from https://github.com/yenchenlin/nerf-pytorch/blob/1f064835d2cca26e4df2d7d130daa39a8cee1795/run_nerf_helpers.py
class Embedder:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.create_embedding_fn()
        
    def create_embedding_fn(self):
        embed_fns = []
        d = self.kwargs['input_dims']
        out_dim = 0
        if self.kwargs['include_input']:
            embed_fns.append(lambda x : x)
            out_dim += d
            
        max_freq = self.kwargs['max_freq_log2']
        N_freqs = self.kwargs['num_freqs']
        
        if self.kwargs['log_sampling']:
            freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
        else:
            freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)
            
        for freq in freq_bands:
            for p_fn in self.kwargs['periodic_fns']:
                embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
                out_dim += d
                    
        self.embed_fns = embed_fns
        self.out_dim = out_dim
        
    def embed(self, inputs):
        return torch.cat([fn(inputs) for fn in self.embed_fns], -1)

def get_embedder(multires):
    embed_kwargs = {
                'include_input' : True,
                'input_dims' : 3,
                'max_freq_log2' : multires-1,
                'num_freqs' : multires,
                'log_sampling' : True,
                'periodic_fns' : [torch.sin, torch.cos],
    }
    
    embedder_obj = Embedder(**embed_kwargs)
    embed = lambda x, eo=embedder_obj : eo.embed(x)
    return embed, embedder_obj.out_dim

In [ ]:
sdf_model = MLP(SDF_MLP_CONFIG).to(device)
print(sdf_model)
print('\n\n')
summary(sdf_model, input_size= tets_verts.shape)

MLP(
  (hiddens): ModuleList(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=64, bias=True)
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1          [-1, 277410, 256]           1,024
            Linear-2          [-1, 277410, 256]          65,792
            Linear-3          [-1, 277410, 128]          32,896
            Linear-4           [-1, 277410, 64]           8,256
            Linear-5            [-1, 277410, 1]              65
Total params: 108,033
Trainable params: 108,033
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 3.17
Forward/backward pass size (MB): 1492.11

Little Test:

In [ ]:
pred_sdfs, f_vs = sdf_model(tets_verts)

print(f'Input grid shape is : {tets_verts.shape}')
print(f'Output shape of the predicted SDFs should be {tets_verts.shape[0], 1} and it actually is {tuple(pred_sdfs.shape)}')
print(f'Output shape of the feature vectors f_vs should be {tets_verts.shape[0], 64} and it actually is {tuple(f_vs.shape)}')
print(pred_sdfs)

Input grid shape is : torch.Size([277410, 3])
Output shape of the predicted SDFs should be (277410, 1) and it actually is (277410, 1)
Output shape of the feature vectors f_vs should be (277410, 64) and it actually is (277410, 64)
tensor([[-0.0267],
        [-0.0269],
        [-0.0267],
        ...,
        [-0.0265],
        [-0.0266],
        [-0.0266]], device='cuda:0', grad_fn=<AddmmBackward0>)


# Set up Optimizer

In [ ]:
sdf_vars = [p for _, p in sdf_model.named_parameters()]
sdf_optimizer = torch.optim.Adam(sdf_vars, lr=lr)
sdf_scheduler = torch.optim.lr_scheduler.LambdaLR(sdf_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

# Training

In [ ]:
# takes in a module and applies the specified weight initialization
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
        values taken from a normal distribution.'''

    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        y = m.in_features
    # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0.0,1/np.sqrt(y))
    # m.bias.data should be 0
        m.bias.data.fill_(0)

In [ ]:
sdf_model.apply(weights_init_normal)

MLP(
  (hiddens): ModuleList(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=64, bias=True)
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
def sdf_train(iterations, model, optimizer, scheduler):
  model.train()
  gt_verts = wt_verts.to(device)
  gt_faces = wt_faces.to(device)

  f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)
  gt_sdfs, idx, t = point_to_mesh_distance(tets_verts.unsqueeze(0), f)

  sign = kaolin.ops.mesh.check_sign(gt_verts.unsqueeze(0).to(device), gt_faces.to(device), tets_verts.unsqueeze(0))
  gt_sdfs[sign==False] *= -1

  for it in range(iterations):
      pred_sdfs, f_vs = model(tets_verts)
      
      loss = F.mse_loss(pred_sdfs, gt_sdfs.squeeze(0).unsqueeze(1), reduction='sum')

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      scheduler.step()

      if (it) % save_every == 0 or it == (iterations - 1): 
          print ('Iteration {} - loss: {}'.format(it, loss))


In [ ]:
# ~12 min. Speed up?
sdf_train(iterations, sdf_model, sdf_optimizer, sdf_scheduler)

Iteration 0 - loss: 5914.7744140625
Iteration 100 - loss: 58.16108703613281
Iteration 200 - loss: 19.72701072692871
Iteration 300 - loss: 6.619785785675049
Iteration 400 - loss: 4.406362533569336
Iteration 500 - loss: 3.4375720024108887
Iteration 600 - loss: 3.3445141315460205
Iteration 700 - loss: 2.316502094268799
Iteration 800 - loss: 2.0546364784240723
Iteration 900 - loss: 1.7777180671691895
Iteration 1000 - loss: 1.7362644672393799
Iteration 1100 - loss: 1.5234479904174805
Iteration 1200 - loss: 1.4100139141082764
Iteration 1300 - loss: 1.37628972530365
Iteration 1400 - loss: 1.2643296718597412
Iteration 1500 - loss: 1.1917943954467773
Iteration 1600 - loss: 1.3606736660003662
Iteration 1700 - loss: 1.0887134075164795
Iteration 1800 - loss: 1.0398125648498535
Iteration 1900 - loss: 1.1708418130874634
Iteration 1999 - loss: 0.9650946855545044


 # Surface refinement utils

In [ ]:
# Get edges lists of shape [E, 2] from face list of shape [V, 4]

def get_edges(input):
  c = torch.combinations(torch.arange(input.size(1)), r=2)
  x = input[:,None].expand(-1,len(c),-1).cpu()
  idx = c[None].expand(len(x), -1, -1)
  x = x.gather(dim=2, index=idx)

  return x.view(-1, *x.shape[2:])

# Extract tets under certain sdf restrictions:
# if thresh = 0, return all surface tetrahedrons
# if thresh > 0, return all tetrahedrons whose vertices' sdfs are all in the range [-thresh, thresh]

def extract_tet(tets, sdf, thresh, non_surf=False):

  assert thresh >= 0

  if thresh == 0:
    mask = sdf[tets] > 0
    mask_int = mask.squeeze(2).long()
    t = mask_int.sum(1)
    surf_tets = tets[(t > 0) & (t < 4)]
  else:
    mask = (sdf[tets] >= -thresh) & (sdf[tets] <= thresh)
    mask_int = mask.squeeze(2).long()
    t = mask_int.sum(1)
    surf_tets = tets[t == 4]

  surf_tets_tuple = surf_tets.unique(return_inverse=True)
  surf_tets_idx, surf_tets = surf_tets_tuple[0], surf_tets_tuple[1]

  if non_surf:
    if thresh == 0:
      non_surf_tets = tets[~((t > 0) & (t < 4))]
    else:
      non_surf_tets = tets[t < 4]
    non_surf_tets_tuple = non_surf_tets.unique(return_inverse=True)
    non_surf_tets_idx, non_surf_tets = non_surf_tets_tuple[0], non_surf_tets_tuple[1]
    return surf_tets_idx, surf_tets, non_surf_tets_idx, non_surf_tets

  return surf_tets_idx, surf_tets

#Get output from initial MLP

def get_pred_sdfs(model, tets_verts):
  model.eval()

  with torch.no_grad():
    pred_sdfs, f_vs = model(tets_verts)

  return pred_sdfs, f_vs

# Get ground truth sdf from input verts and faces
# input shape: [batch_size, num_vertices, 3], [num_faces, 4], [batch_size, num_points, 3]
# output shape: [num_points, 1]

def get_gt_sdfs(gt_verts, gt_faces, points, f=None):

  if f == None:
    f = index_vertices_by_faces(gt_verts, gt_faces)
  d,_,_ = point_to_mesh_distance(points, f)

  s = kaolin.ops.mesh.check_sign(gt_verts, gt_faces, points)
  d[s==True] *= -1

  return d.squeeze(0).unsqueeze(1)

# Check quality of sdf prediction
Test `extract_tet` here. Most ideal if about 20k~30k surface vertices are extracted.

In [ ]:
d = get_gt_sdfs(wt_verts.unsqueeze(0).to(device), wt_faces.to(device), tets_verts.unsqueeze(0))
pred_sdfs, _ = get_pred_sdfs(sdf_model, tets_verts)

# Surface refinement model

In [ ]:
"""
Graph-res net:
Identify surface tetrahedral, build adj matrix, 
"""

from tqdm import tqdm

from torch import nn
from torch.nn import functional as F
from pytorch3d.ops import GraphConv

# a single res block layer with dimension 256 & 128
class GResBlock(nn.Module): 
    def __init__(self, in_dim, hidden_dim, activation=None):
        super(GResBlock, self).__init__()

        self.conv1 = GraphConv(in_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, in_dim)
        self.activation = F.relu if activation else None
    
    def forward(self, inputs):
        input, adj = inputs[0], inputs[1]
        x = self.conv1(input, adj)
        if self.activation:
          x = self.activation(x)
        x = self.conv2(x, adj)
        if self.activation:
          x = self.activation(input + x)
        
        return [x, adj]

class GBottleneck(nn.Module):
    def __init__(self, block_num, in_dim, hidden_dim, out_dim, activation=None):
        super(GBottleneck, self).__init__()

        resblock_layers = [GResBlock(in_dim=hidden_dim[0], hidden_dim=hidden_dim[1], activation=activation)
                          for _ in range(block_num)]
        self.blocks = nn.Sequential(*resblock_layers)
        self.conv1 = GraphConv(in_dim, hidden_dim[0])

        self.activation = F.relu if activation else None
    
    def forward(self, inputs, adj):
        x = self.conv1(inputs, adj)
        if self.activation:
          x = self.activation(x)
        x = self.blocks([x, adj])[0]
        if self.activation:
          x = self.activation(x)

        return x

class GCN_Res(nn.Module):
    def __init__(self, config):
        super(GCN_Res, self).__init__()

        self.in_dim = config['in_dim']
        self.hidden_dim = config['hidden_dim']
        self.out_dim = config['out_dim']
        self.activation = config['activation']
        self.mlp_hdim = config['mlp_hdim']
        self.mlp_odim = config['mlp_odim']

        self.gcn_res = nn.ModuleList([GBottleneck(2, self.in_dim, self.hidden_dim, self.out_dim, self.activation)])

        self.sdf_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], 1, bias=False),
        )

        self.deform_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], 3, bias=False),
        )

        self.feature_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], self.mlp_hdim[1], bias=False),
        )
        


    def forward(self, inputs, adj):

        x = self.gcn_res[0](inputs, adj)

        sdf = self.sdf_mlp(x)
        deform = self.deform_mlp(x)
        deform = torch.tanh(deform)
        feature = self.feature_mlp(x)
        
        return sdf, deform, feature


In [ ]:
def laplace_regularizer_const(mesh_verts, mesh_faces):
    term = torch.zeros_like(mesh_verts)
    norm = torch.zeros_like(mesh_verts[..., 0:1])

    v0 = mesh_verts[mesh_faces[:, 0], :]
    v1 = mesh_verts[mesh_faces[:, 1], :]
    v2 = mesh_verts[mesh_faces[:, 2], :]

    term.scatter_add_(0, mesh_faces[:, 0:1].repeat(1,3), (v1 - v0) + (v2 - v0))
    term.scatter_add_(0, mesh_faces[:, 1:2].repeat(1,3), (v0 - v1) + (v2 - v1))
    term.scatter_add_(0, mesh_faces[:, 2:3].repeat(1,3), (v0 - v2) + (v1 - v2))

    two = torch.ones_like(v0) * 2.0
    norm.scatter_add_(0, mesh_faces[:, 0:1], two)
    norm.scatter_add_(0, mesh_faces[:, 1:2], two)
    norm.scatter_add_(0, mesh_faces[:, 2:3], two)

    term = term / torch.clamp(norm, min=1.0)

    return torch.mean(term**2)

def gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it):

    #surface alignment loss
    m = pytorch3d.structures.Meshes([mesh_verts, gt_verts], [mesh_faces, gt_faces])
  
    # pc = pytorch3d.ops.sample_points_from_meshes(m, num_samples=50000)
    if mesh_verts.shape[0] > 0:
      pred_points = kaolin.ops.mesh.sample_points(mesh_verts.unsqueeze(0), mesh_faces, 50000)[0][0]
      gt_points = kaolin.ops.mesh.sample_points(gt_verts.unsqueeze(0), gt_faces, 50000)[0][0]
      # pred_points,gt_points = pc[0], pc[1]
      chamfer = kaolin.metrics.pointcloud.chamfer_distance(pred_points.unsqueeze(0), gt_points.unsqueeze(0), squared=False).mean()
    else:
      chamfer = 0


    #chamfer, p_norms = pytorch3d.loss.chamfer_distance(pred_points.unsqueeze(0), gt_points.unsqueeze(0), point_reduction='sum')
    # norm = torch.sum(1-torch.abs(p_norms))
    # norm = pytorch3d.loss.mesh_normal_consistency(m)
    if it > iterations//2:
      lap = laplace_regularizer_const(mesh_verts, mesh_faces)
      return 500*chamfer + lap * laplacian_weight #+ 1e-6*norm
    return 500*chamfer #+ 1e-6*norm 

# Train GCN Model for Surface Refinement
The model is structured following Curve-GCN that has two Graph Res Block with three MLPs, outputs sdf residual, deform residual, updated feature vector, respectively. 

Before start training, need to run the model with `gcn_pretrain` to ensure that model does not run on random surface. 

Pretrain on sphere is not tested and not guaranteed to be working




In [ ]:
CONFIG_GCNRES = {
    'in_dim': 68,
    'hidden_dim': [128, 256],
    'out_dim': 128,
    'activation': True,
    'mlp_hdim': [128,64],
    'mlp_odim': 68,
}

# Same set of Hyperparam is applied
lr = 1e-3
laplacian_weight = 0.1
gcn_iterations = 5000
save_every = 100
multires = 2
grid_res = 128

In [ ]:
from tqdm import tqdm

def gcn_pretrain(iterations, gcn_model, optimizer, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
  gcn_model.train()

  assert thresh >= 0

  surf_tets_verts_idx, surf_tets_faces = extract_tet(tets, pred_sdf, thresh)
  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])
  surf_sdfs = pred_sdf[surf_tets_verts_idx]
  surf_tets_edges = torch.clone(get_edges(surf_tets_faces).to(device))
  
  gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)

  gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)

  timelapse.add_mesh_batch(
          category='tet_surface',
          vertices_list=[surf_tets_verts.cpu()],
          faces_list=[surf_tets_faces.cpu()]
      )

  for it in tqdm(range(iterations)):
    verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

    sdf, deform, fv = gcn_model(verts_f, surf_tets_edges)

    update_verts = surf_tets_verts + deform / grid_res
    update_sdfs = surf_sdfs + sdf

    gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, update_verts.unsqueeze(0), gt_f)

    sdf_loss = F.mse_loss(sdf, gt_sdfs - surf_sdfs, reduction='sum')

    optimizer.zero_grad()
    sdf_loss.backward(retain_graph=True)
    optimizer.step()

    if it == iterations-1:
      print('Iteration {} - loss: {}'.format(iterations, sdf_loss))


In [ ]:
"""
Identify surface Tetrahedral
input: f = [v, s, F_vol, f_64]
output: f = [delta v, delta s, f_64]

update v = v + delta v, s = s + delta s
generate new meshes by marching tet from v and s; calculate loss

loss = chamfer + sdf_loss + deform_loss
"""

def gcn_train(iterations, gcn_model, optimizer, scheduler, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
  gcn_model.train()

  assert thresh >= 0

  """

  Firstly, based on predicted sdf value, extract out all surface tetrahedrons
  i.e. 1-3 vertices of tetrahedrons are with different sdf signs
  return value: 
  --> surf_tets_idx: origin index of vertices as in tet grid
  --> surf_tets_faces: reindexed faces list start from 0

  """

  surf_tets_verts_idx, surf_tets_faces = extract_tet(tets, pred_sdf, thresh)
  surf_tets_edges = get_edges(surf_tets_faces).to(device)

  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

  # obtain per vertex features: 
  # sdf, position, feature vector

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])
  surf_sdfs = torch.clone(pred_sdf[surf_tets_verts_idx])
  surf_verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

  # calculate ground truth sdf value over all vertices in tet mesh

  # gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces).to(device)
  # gt_sdfs, _, _ = point_to_mesh_distance(tets_verts.unsqueeze(0), gt_f)
  # gt_sdfs = gt_sdfs.squeeze(0).unsqueeze(1)

  gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)

  gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)
  
  gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, tets_verts.unsqueeze(0), gt_f)

  for it in range(iterations):
    
    """
    
    Secondly, predict with a 2-layer GCN followed by 2-layer MLP
    output:
    --> [delta v, delta s, new f_s] \in R^68

    """

    sdf, deform, fv = gcn_model(surf_verts_f, surf_tets_edges)
    
    """

    Update surface position, sdf, and f_s

    """

    #updated sdf

    update_sdfs = torch.clone(pred_sdf)
    update_sdfs[surf_tets_verts_idx] += sdf
    

    #update vertices positions

    vd = deform
    update_tets_verts = torch.clone(tets_verts)
    update_tets_verts[surf_tets_verts_idx] += deform / grid_res

    #update vertices features

    vfvs = fv.detach()
    update_tets_f = torch.clone(tets_verts_features)
    update_tets_f[surf_tets_verts_idx] += vfvs

    #test
    MT_sdf = surf_sdfs + sdf
    MT_verts = surf_tets_verts + deform

    """

    Marching Tetrahedra based on new sdf value and deformed vertices in the tet grid

    """

    mesh_verts, mesh_faces = kaolin.ops.conversions.marching_tetrahedra(update_tets_verts.unsqueeze(0), tets, update_sdfs.squeeze(1).unsqueeze(0))
    mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]

    """

    Compute Loss for First surface refinement: 
    Normal consistency + surface alignment + laplacian smooth + sdf L2-reg + deform L2-reg

    """

    # L2 sdf reg: 

    # Calculate SDF non_surface tetrahedron in newly generated mesh

    # ns_mask = torch.ones(tets_verts.shape[0])
    # ns_mask[surf_tets_verts_idx] = 0
    # non_surf_verts = torch.clone(tets_verts[ns_mask == 1])

    # p_sdfs = get_gt_sdfs(mesh_verts.unsqueeze(0), mesh_faces, non_surf_verts.unsqueeze(0))

    # # Get ground truth SDF value of non_surface tetrahedron vertices and truncate at +- 0.3 to focus on surface

    # ns_g = gt_sdfs[ns_mask == 1]
    # mask = (ns_g >= -0.1) & (ns_g <= 0.1)
    # p = p_sdfs[mask]
    # g = ns_g[mask]

    # Also calculate surface sdf loss

    s_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, update_tets_verts[surf_tets_verts_idx].unsqueeze(0), gt_f)

    # sdf_loss = F.mse_loss(update_sdfs, s_sdfs, reduction='sum') #+ F.mse_loss(p.to(device), g.to(device), reduction='mean')

    sdf_loss = F.mse_loss(sdf, s_sdfs-surf_sdfs, reduction='mean') #+ F.mse_loss(p.to(device), g.to(device), reduction='mean')

    #L2 deform reg

    # deform_loss = F.mse_loss(update_tets_verts, tets_verts, reduction='sum')
    deform_loss = F.mse_loss(deform, torch.zeros(deform.shape).to(device), reduction='mean')

    #surface alignment loss

    r_loss = gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it)

    g_loss = r_loss + deform_loss + 0.4*sdf_loss

    optimizer.zero_grad()
    torch.autograd.set_detect_anomaly(True)
    # r_loss.backward(retain_graph=True)
    # deform_loss.backward(retain_graph=True)
    # sdf_loss.backward(retain_graph=True)
    g_loss.backward(retain_graph=True)
    optimizer.step()
    scheduler.step()

    if (it) % save_every == 0 or it == (iterations - 1): 
      print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, g_loss, mesh_verts.shape[0], mesh_faces.shape[0]))
      
      # save reconstructed mesh
      timelapse.add_mesh_batch(
          iteration=it+1,
          category='train_res',
          vertices_list=[mesh_verts.cpu()],
          faces_list=[mesh_faces.cpu()]
      )

In [ ]:
refine_model = GCN_Res(CONFIG_GCNRES).to(device)

print(refine_model)

pre_vars = [p for _, p in refine_model.named_parameters()]
pre_optimizer = torch.optim.Adam(pre_vars, lr=lr)

GCN_Res(
  (gcn_res): ModuleList(
    (0): GBottleneck(
      (blocks): Sequential(
        (0): GResBlock(
          (conv1): GraphConv(128 -> 256, directed=False)
          (conv2): GraphConv(256 -> 128, directed=False)
        )
        (1): GResBlock(
          (conv1): GraphConv(128 -> 256, directed=False)
          (conv2): GraphConv(256 -> 128, directed=False)
        )
      )
      (conv1): GraphConv(68 -> 128, directed=False)
    )
  )
  (sdf_mlp): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=False)
    (1): Linear(in_features=128, out_features=64, bias=False)
    (2): Linear(in_features=64, out_features=1, bias=False)
  )
  (deform_mlp): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=False)
    (1): Linear(in_features=128, out_features=64, bias=False)
    (2): Linear(in_features=64, out_features=3, bias=False)
  )
  (feature_mlp): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=False)
    (1): Linear(in_features

In [ ]:
gcn_pretrain(500, refine_model, pre_optimizer, d, tets_verts, tets, f_vs, sample_verts, sample_faces, 0.001)

In [ ]:
refine_vars = [p for _, p in refine_model.named_parameters()]
refine_optimizer = torch.optim.Adam(pre_vars, lr=lr)
refine_scheduler = torch.optim.lr_scheduler.LambdaLR(pre_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

In [ ]:
torch.cuda.empty_cache()
gcn_train(gcn_iterations, refine_model, refine_optimizer, refine_scheduler, d, tets_verts, tets, f_vs, sample_verts, sample_faces, 0.001)

# Visualization

In [ ]:
#Use pyngrok to access localhost:80 on Colab

!pip install pyngrok --quiet 
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2IGctyaa9n7vRBd8qq7pzd0bNKh_2pDFmKRk5Af1QDq295xZ4"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

     |████████████████████████████████| 761 kB 4.1 MB/s 


In [ ]:
#generating a public url mapped to localhost 80
public_url = ngrok.connect(port=80, proto="http", options={"bind_tls": True, "local": True})
print("Tracking URL:", public_url)

Tracking URL: NgrokTunnel: "http://24af-35-201-250-22.ngrok.io" -> "http://localhost:80"


In [ ]:
#Start Kaolin Dash3D on localhost:80 
!kaolin-dash3d --logdir=/content/drive/MyDrive/CV_DMTet/Logs --port=80

Dash3D server starting. Go to: http://localhost:80
2022-12-11 22:10:10,329|    INFO|kaolin.visualize.timelapse| No checkpoints found for type pointcloud: no files matched pattern pointcloud*.usd in /content/drive/MyDrive/CV_DMTet/Logs
2022-12-11 22:10:10,335|    INFO|kaolin.visualize.timelapse| No checkpoints found for type voxelgrid: no files matched pattern voxelgrid*.usd in /content/drive/MyDrive/CV_DMTet/Logs
2022-12-11 22:10:28,353|    INFO|kaolin.visualize.timelapse| No checkpoints found for type pointcloud: no files matched pattern pointcloud*.usd in /content/drive/MyDrive/CV_DMTet/Logs
2022-12-11 22:10:28,359|    INFO|kaolin.visualize.timelapse| No checkpoints found for type voxelgrid: no files matched pattern voxelgrid*.usd in /content/drive/MyDrive/CV_DMTet/Logs
2022-12-11 22:10:31,251|    INFO| tornado.access| 200 GET / (127.0.0.1) 2911.40ms
2022-12-11 22:10:32,088|    INFO| tornado.access| 200 GET /static/thirdparty.css (127.0.0.1) 615.87ms
2022-12-11 22:10:32,812|    INFO|